In [1]:
import os
import sys
os.chdir('../')
sys.path.append(os.path.join(os.getcwd(), "src"))

In [2]:
from dataclasses import dataclass
from pathlib import Path
from WattPredictor.utils.helpers import *
from WattPredictor.utils.exception import *
from WattPredictor.constants import *
from WattPredictor import logger

In [3]:
@dataclass
class ModelEvaluationConfig:
    model_path: Path
    cutoff_date: str
    input_seq_len: int
    step_size: int
    metrics_path: Path

@dataclass(frozen=True)
class FeatureStoreConfig:
    hopsworks_project_name: str
    hopsworks_api_key: str

In [ ]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_PATH,
                 params_filepath = PARAMS_PATH,
                 schema_filepath = SCHEMA_PATH):
        
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.training

        create_directories([config.root_dir])
        
        model_evaluation_config =  ModelEvaluationConfig(
            model_path=Path(config.model_path),
            cutoff_date=params.cutoff_date,
            input_seq_len= params.input_seq_len,
            step_size = params.step_size,
            metrics_path=Path(config.metrics_path)
        )

        return model_evaluation_config
    
    def get_feature_store_config(self) -> FeatureStoreConfig:

        config = self.config.feature_store

        feature_store_config = FeatureStoreConfig(
                hopsworks_project_name=config.hopsworks_project_name,
                hopsworks_api_key=os.environ['hopsworks_api_key'],
        )

        return feature_store_config

In [5]:
import hopsworks
import pandas as pd
import sys
from WattPredictor.utils.exception import CustomException
from WattPredictor import logger


class FeatureStore:
    def __init__(self, config):
        self.config = config
        self.connect()

    def connect(self):
        try:
            self.project = hopsworks.login(
                project=self.config.hopsworks_project_name,
                api_key_value=self.config.hopsworks_api_key
            )
            self.feature_store = self.project.get_feature_store()
            self.dataset_api = self.project.get_dataset_api()
            logger.info(f"Connected to Hopsworks Feature Store: {self.config.hopsworks_project_name}")
        except Exception as e:
            raise CustomException(e, sys)

    def create_or_update_feature_group(self, name, df, primary_key, event_time, description, online=False):
        """
        Alternative method that handles feature group creation/update more robustly
        """
        try:
            import time
            
            fg = None
            try:
                fg = self.feature_store.get_feature_group(name=name, version=1)
                
                if fg is not None:
                    logger.info(f"Feature Group '{name}' found. Will insert new data...")
                    fg.insert(df, write_options={"wait_for_job": True})
                    logger.info(f"Data successfully inserted into existing Feature Group '{name}'.")
                    return fg
                else:
                    logger.info(f"Feature Group '{name}' returned None, treating as not found.")
                    raise Exception("Feature group not found")
                    
            except Exception as get_error:
                logger.info(f"Feature Group '{name}' not found: {str(get_error)}")
                logger.info("Proceeding to create new feature group...")

            try:
                if hasattr(self.feature_store, 'get_or_create_feature_group'):
                    fg = self.feature_store.get_or_create_feature_group(
                        name=name,
                        version=1,
                        primary_key=primary_key,
                        event_time=event_time,
                        description=description,
                        online_enabled=online
                    )
                    logger.info(f"Feature Group '{name}' created using get_or_create_feature_group")
                else:
                    fg = self.feature_store.create_feature_group(
                        name=name,
                        version=1,
                        primary_key=primary_key,
                        event_time=event_time,
                        description=description,
                        online_enabled=online
                    )
                    logger.info(f"Feature Group '{name}' created using create_feature_group")

            except Exception as create_error:
                logger.error(f"Failed to create feature group '{name}': {str(create_error)}")
                raise create_error

            if fg is None:
                raise Exception(f"Feature group '{name}' is None after creation")

            logger.info(f"Inserting {len(df)} rows into Feature Group '{name}'...")
            fg.insert(df, write_options={"wait_for_job": True})
            logger.info(f"Data successfully inserted into Feature Group '{name}'.")
            
            return fg

        except Exception as e:
            logger.error(f"Error in create_or_update_feature_group for '{name}': {str(e)}")
            raise CustomException(e, sys)

    def create_feature_view(self, name, feature_group_name, features):
        try:
            try:
                existing_fv = self.feature_store.get_feature_view(name=name, version=1)
                if existing_fv is not None:
                    existing_fv.delete()
                    logger.info(f"Deleted existing Feature View '{name}' for clean recreation.")
            except Exception as delete_error:
                logger.warning(f"No existing Feature View to delete: {delete_error}")

            fg = self.feature_store.get_feature_group(name=feature_group_name, version=1)
            query = fg.select(features)
            fv = self.feature_store.create_feature_view(
                name=name,
                version=1,
                query=query,
                description=f"Feature View for {feature_group_name}"
            )
            logger.info(f"Feature View '{name}' created successfully.")
            return fv

        except Exception as e:
            raise CustomException(e, sys)

    def save_training_dataset(self, feature_view_name, version_description, output_format="csv"):
        try:
            fv = self.feature_store.get_feature_view(name=feature_view_name, version=1)
            td = fv.create_training_data(
                description=version_description,
                data_format=output_format,
                write_options={"wait_for_job": True}
            )
            logger.info(f"Training dataset version for Feature View '{feature_view_name}' created.")
            return td
        except Exception as e:
            raise CustomException(e, sys)

    def load_latest_training_dataset(self, feature_view_name):
        try:
            fv = self.feature_store.get_feature_view(name=feature_view_name, version=1)
            return fv.training_data()
        except Exception as e:
            raise CustomException(e, sys)

    def get_online_features(self, feature_view_name, key_dict: dict):
        try:
            fv = self.feature_store.get_feature_view(name=feature_view_name, version=1)
            return fv.get_online_features(key_dict)
        except Exception as e:
            raise CustomException(e, sys)

    def upload_file_safely(self, local_path: str, target_name: str):
        try:
            self.dataset_api.upload(
                local_path,
                f"Resources/wattpredictor_artifacts/{target_name}",
                overwrite=True
            )
            logger.info(f"Uploaded file to Feature Store: {target_name}")
        except Exception as e:
            raise CustomException(e, sys)

In [6]:
import os
import sys
import json
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score,root_mean_squared_error
from WattPredictor.utils.helpers import create_directories, save_json
from WattPredictor.utils.exception import CustomException
from WattPredictor import logger
from WattPredictor.utils.ts_generator import features_and_target


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig, feature_store_config):
        self.config = config
        self.feature_store = FeatureStore(feature_store_config)

    def evaluate(self):
        try:
            df, _ = self.feature_store.load_latest_training_dataset("elec_wx_features_view")
            df = df[['date', 'demand', 'sub_region_code', 'temperature_2m']]
            df.sort_values("date", inplace=True)
            _, test_df = df[df['date'] >= self.config.cutoff_date], df[df['date'] < self.config.cutoff_date]

            test_x, test_y = features_and_target(test_df, input_seq_len=self.config.input_seq_len, step_size=self.config.step_size)
            test_x.drop(columns=["date"], errors="ignore", inplace=True)

            # Load model from Hopsworks
            model_registry = self.feature_store.project.get_model_registry()
            model = model_registry.get_model("wattpredictor_xgboost", version=1)
            model_dir = model.download()
            model_path = os.path.join(model_dir, "model.joblib")
            model_instance = joblib.load(model_path)


            preds = model_instance.predict(test_x)

            metrics = {
                "mse": mean_squared_error(test_y, preds),
                "mae": mean_absolute_error(test_y, preds),
                "rmse": root_mean_squared_error(test_y, preds),
                "mape": np.mean(np.abs((test_y - preds) / test_y)) * 100 if np.any(test_y != 0) else np.inf,
                "r2_score": r2_score(test_y, preds),
                "adjusted_r2": 1 - (1 - r2_score(test_y, preds)) * (len(test_y) - 1) / (len(test_y) - test_x.shape[1] - 1)
            }

            create_directories([os.path.dirname(self.config.metrics_path)])
            save_json(self.config.metrics_path, metrics)

            logger.info(f"Evaluation complete. Metrics: {metrics}")
            return metrics

        except Exception as e:
            raise CustomException(e, sys)


In [7]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    feature_store_config = config.get_feature_store_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config, feature_store_config=feature_store_config)
    model_evaluation.evaluate()
except Exception as e:
    raise CustomException(str(e), sys)

[2025-07-14 10:58:49,167: INFO: helpers: yaml file: config_file\config.yaml loaded successfully]
[2025-07-14 10:58:49,181: INFO: helpers: yaml file: config_file\schema.yaml loaded successfully]
[2025-07-14 10:58:49,186: INFO: helpers: yaml file: config_file\params.yaml loaded successfully]
[2025-07-14 10:58:49,186: INFO: helpers: created directory at: artifacts]
[2025-07-14 10:58:49,192: INFO: external: Initializing external client]
[2025-07-14 10:58:49,194: INFO: external: Base URL: https://c.app.hopsworks.ai:443]
[2025-07-14 10:58:53,942: INFO: python: Python Engine initialized.]

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1240214
[2025-07-14 10:58:56,745: INFO: 4147405043: Connected to Hopsworks Feature Store: WattPredictor]
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.27s) 
[2025-07-14 10:59:06,013: WARNING: warnings: VersionWarning: Incremented version to `3`.
]


Generating TS features: 100%|██████████| 11/11 [00:00<00:00, 24.73it/s]


Downloading: 0.000%|          | 0/170165 elapsed<00:00 remaining<?

[2025-07-14 10:59:10,764: INFO: helpers: created directory at: artifacts\model_evaluation]
[2025-07-14 10:59:10,766: INFO: helpers: json file saved at: artifacts\model_evaluation\metrics.json]
[2025-07-14 10:59:10,767: INFO: 4253362285: Evaluation complete. Metrics: {'mse': 229.4557215666434, 'mae': 11.6898894522703, 'rmse': 15.147795930980962, 'mape': 1.04106801796017, 'r2_score': 0.9998809553708564, 'adjusted_r2': 0.9998077472686276}]
